# A notebook to email geocomp grads the kata solutions

This is based on my [email from python](Email_from_python.ipynb) notebook which is based on [Corey Schafer's video](https://www.youtube.com/watch?v=JRCJ6RtE3xU&t=751s).

For the Google API, I used both [Google's docs](https://developers.google.com/sheets/api/guides/values) and [this medium article](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c) by [Prafulla Dalvi](https://medium.com/@prafuld3).

In [1]:
#email imports
import smtplib
from email.message import EmailMessage
from enviro import gmail_login, gmail_pwd, SPREADSHEET_ID, SPREADSHEET_ID_TESTING, test_emails
#Google API imports
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
#General imports
import glob
import pandas as pd
from datetime import date

In [2]:
### GOOGLE API GLOBALS
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

# The ID and range of a sample spreadsheet.
#SPREADSHEET_ID_TESTING: imported from enviro
#SPREADSHEET_ID: imported from enviro
RANGE_NAME = 'Sheet1'

In [3]:
# set up variables
project_path = '../../../weekly-challenge/'
login = gmail_login
password = gmail_pwd

In [4]:
def connect_to_google_sheets_API():
    """
    Connect to the Google Sheet API, verify credits and return a service object.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    return service

In [5]:
def connect_to_google_sheets_API_write():
    """
    Connect to the Google Sheet API, verify credits and return a service object.
    """
    creds = None
    if os.path.exists('token_write.pickle'):
        with open('token_write.pickle', 'rb') as token:
            creds = pickle.load(token)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token_write.pickle', 'wb') as token:
            pickle.dump(creds, token)

    try:
        service = build('sheets', 'v4', credentials=creds)
        print('sheets', 'service created successfully')
        return service
    except Exception as e:
        print(e)
        return None

In [6]:
def read_from_google_sheet(service):
    """Call the Sheets API"""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
    values = result.get('values', [])

    if not values:
        print('No data found, aborting.')
        return None
    else:
        df = pd.DataFrame.from_records(values, columns=values[0])
        df = df.drop([0])
        df = df.reset_index(drop=True)

    return df

In [7]:
def write_to_google_sheet_log_file(df, service):
    """Write date back to Google Sheet log file for given class_abr and challenge"""
    response_date = service.spreadsheets().values().update(spreadsheetId=SPREADSHEET_ID,
                                                           valueInputOption='RAW',
                                                           range=RANGE_NAME,
                                                           body=dict(
                                                               majorDimension='ROWS',
                                                               values=df.T.reset_index().T.values.tolist()
                                                           )
                                                          ).execute()
    print('Sheet successfully Updated')
    return None

In [8]:
def get_recipient_names_and_emails(class_abr, project_path=None):
    """
    Given a class abreviation, return a dict of names and emails.
    args:
        class_abr: `str` class abreviation for example: `apr20` or `mar20`
    kwargs:
        project_path: `str` or None path prefix to search locally, for example '../../../weekly-challenge/'
    """
    if project_path:
        df = pd.read_csv(project_path + 'proj-kata_email_list.csv')
    else:
        df = read_from_google_sheet(connect_to_google_sheets_API())
    df = df.loc[df['class_abr'] == class_abr]
    if len(df['class_abr'] == class_abr) == 0:
        print('The class_abr you provided does not exist or was misspelled.')
        return None
    
    return pd.Series(df.email.values, index=df.recipient).to_dict()

In [9]:
def get_challenge_solution(challenge, project_path):
    """Given a challenge name, return the corresponding path to the solution file."""
    challenge = challenge.split('-')[0].title() + '-' + challenge.split('-')[1] if '-' in challenge else challenge.title()
        
    try:
        solution = glob.glob(f'{project_path}notebooks/{challenge}_solution.ipynb')[0]
    except IndexError:
        print('The challenge you provided does not exist or was misspelled.')
        return None
    
    return solution

In [10]:
def send_kata_solution(challenge, recipient, msg_to, msg_from='robert@agilescientific.com'):
    """
    Send an email with the kata solution as an attachment to the recipient, by name.
    
    args:
        challenge: `str` name of the kata challenge
        recipient: `str` name of the recipient
        msg_to: `str` valid email address of the `recipient` to send the message to
    kwargs:
        msg_from: `str` email from which to send the email - this is only for the reply-to field.
        The email is actually sent from my personal-work email
    Returns:
        None
    """

    msg = EmailMessage()
    msg['Subject'] = f'kata.geosci solution for challenge: {challenge}'
    msg['From'] = msg_from
    msg['To'] = msg_to
    msg['Cc'] = msg_from
    msg.set_content(f"""Dear {recipient}, 

I am enclosing the solutions to the {challenge} challenge. I would recommend that you only use this if you're really stuck, or, once you've finished, to see some insights included in the solution.

You can get the next challenge by completing this one !

I'd also like to point you towards Matt's blog about this: 
https://agilescientific.com/blog/2020/4/16/geoscientist-challenge-thyself

and remember you can also get help here https://help.agilescientific.com/home

Finally we'd love to hear your feedback about these challenges. Are they fun? are they useful? are they hard enough or too hard?

And of course, you can unsubscribe at any time by replying to me.

Best wishes,
Rob

Robert Leckenby, PhD
Geoscientist at Agile*
agilescientific.com

PS: this email was sent with Python, because why not ;-)
    """)

    with open(get_challenge_solution(challenge, project_path), 'rb') as f:
        file_data = f.read()
        file_name = f.name.split('/')[-1]
   
    msg.add_attachment(file_data, maintype='text', subtype='ipynb', filename=file_name)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(login, password)
        # ******** WARNING ******** 
        # ******** WARNING ******** 
        # ******** WARNING ******** 
        # the next line will send an actual email
        smtp.send_message(msg)
        #print('dry run, no emails sent')
        #pass
    
    # logging
    print(f'On {date.today()} the *{challenge}* challenge was sent to:\n{msg_to}\n')
    
    return

In [11]:
def make_df(class_abr, challenge, date):
    df = read_from_google_sheet(connect_to_google_sheets_API())
    
    column = get_challenge_solution(challenge, project_path).split('/')[-1]
    df.loc[df['class_abr'] == class_abr, column] = str(date)
    df = df.fillna('')
    return df

In [12]:
def send_emails(class_abr, challenge, project_path, test=False, dry_run=True):
    """Send emails to students with a given challenge"""
    if test:
        class_abr = None
        challenge = 'sample-names'
        emails_test = test_emails # imported from enviro
        for recipient, email in emails_test.items():
            send_kata_solution(challenge, recipient, email)
            
        print(f'Check sending email account ({login}) for errors.\n')
        print('Done =============================================\n')

        return
            
    if dry_run:
        print('Dry run only. Emails would have been sent to these recipients and these emails:')
        print('###############################################################################')  
        for recipient, email in get_recipient_names_and_emails(class_abr).items():
            print(f'recipient: {recipient}\temail: {email}')
        print('End.')
        
        return
    
    else:
        if (get_recipient_names_and_emails(class_abr) == None) or (get_challenge_solution(challenge, project_path) == None):
            print('aborting.')
            return
            
        for recipient, email in get_recipient_names_and_emails(class_abr).items():
            send_kata_solution(challenge, recipient, email)
    
        #write_to_log_file(class_abr, challenge, date.today(), project_path)
        write_to_google_sheet_log_file(make_df(class_abr, challenge, date.today()), connect_to_google_sheets_API_write())
        
    print(f'Check sending email account ({login}) for errors.\n')
    print('Done =============================================\n')

    return

In [13]:
# TESTING CELL
#send_emails('apr20', 'test', project_path=project_path, dry_run=False)